In [1]:
import qctoolkit as qtk
import numpy as np
import glob
import os, re
import copy

/home/samio/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
cyl_map = {
    'si': [
        ['si', 5.431, [14, 14]],
        ['c', 3.566, [6, 6]],
        ['sic', 4.3596, [14, 6]],
        ['cge', 5.574, [6, 32]],
        ['ge', 5.658, [32, 32]],
        ['sige', 5.432, [14, 32]],
        ['sn', 6.4892, [50, 50]],
        ['snsi', 5.961, [14, 50]],
        ['gesn', 6.0758, [32, 50]],
    ],
    'gaas': [
        ['gaas', 5.6535, [31, 33]],
        ['alp', 5.4635, [13, 15]],
        ['alas', 5.660, [13, 33]],
        ['alsb', 6.1355, [13, 51]],
        ['gap', 5.451, [31, 15]],
        ['gasb', 6.09, [31, 51]],
        ['inp', 5.86, [49, 15]],
        ['inas', 6.05, [49, 33]],
        ['insb', 6.47, [49, 51]],
    ],
    'cdse': [
        ['cdse', 6.08, [48, 34]],
        ['zns', 5.41, [30, 16]],
        ['znse', 5.67, [30, 34]],
        ['znte', 6.1, [30, 52]],
        ['cds', 5.82, [48,16]],
        ['cdte', 6.48, [48, 52]],
        ['hgs', 5.8517, [80, 16]],
        ['hgse', 6.085, [80, 34]],
        ['hgte', 6.453, [80, 52]],
    ]
}

# pure GGA with insuite restart

In [5]:
qmsetting = {
    'program': 'abinit',
    'kmesh': [6,6,6],
    'band_scan': [
     [17, 17, 8, 10, 15],
     [[0.25, 0.75, 0.5], # W
      [0.0, 0.0, 0.0],   # Gamma
      [0.0, 0.5, 0.5],   # X
      [0.25, 0.75, 0.5], # L
      [0.5, 0.5, 0.5],   # W
      [0.0, 0.0, 0.0],   # Gamma
     ]],
    'wf_convergence': 1E5,
    'ks_states': 10,
    'link_dep': True, 
    'threads': 1,
}

In [6]:
# construct crystals based on cyl_map and si, gaas, cdse crystals

mol_base = []
for cyl in ['si', 'gaas', 'cdse']:
    name = 'xyz/%s.xyz' % cyl
    mol_base.append(qtk.Molecule(name))
mol_base

mols_grp = []
for mol in mol_base:
    mol_grp = [mol]
    for info in cyl_map[mol.name][1:]:
        new = mol.copy()
        new.name = info[0]
        new.setCelldm([info[1], info[1], info[1], 0, 0, 0])
        for i in range(new.N):
            if new.Z[i] != info[2][i]:
                new.setAtoms(i, Z=info[2][i])
        mol_grp.append(new)
    mols_grp.append(mol_grp)
    
mols_all = list(qtk.flatten(mols_grp))

In [23]:
inps = []
for mols in mols_grp:
    for mol_ref in mols:
        for mol_tar in mols:
            #if mol_tar.name != mol_ref.name:
            #print mol_ref.name + '->' + mol_tar.name
            mol = mol_tar.copy()
            mol.name = mol_tar.name + '_' + mol_ref.name
            ref_name = mol_ref.name + '_a-' + mol_tar.name
            ref_path = os.path.abspath('Eg_ref/'+ref_name)
            print ref_path
            inp = qtk.QMInp(
                mol, 
                restart_wavefunction_path = ref_path,
                restart_density_path = ref_path,
                **qmsetting)
            inps.append(inp)
print len(inps)

/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/si_a-si
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/si_a-c
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/si_a-sic
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/si_a-cge
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/si_a-ge
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/si_a-sige
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/c_a-si
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/c_a-c
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/c_a-sic
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/c_a-cge
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/c_a-ge
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/c_a-sige
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_qtkTest/Eg_ref/sic_a-si
/home/samio/Works/PhD/projects/01_AlGaAs/60_Eg/02_

# HSE06, fresh restart

In [3]:
qmsetting_base = {
    'program': 'abinit',
    'wf_convergence': 1E5,
    'ks_states': 10,
    'link_dep': True, 
    'threads': 1,
    'restart': True,
    'restart_density': True,
}

qmsetting_bnd = copy.deepcopy(qmsetting_base)
qmsetting_bnd['band_scan'] = [
     [15, 20],
     [[0.5, 0.0, 0.25],  # L
      [0.0, 0.0, 0.0],   # Gamma
      [0.0, 0.5, 0.5],   # X
     ]]

qmsetting_dos = copy.deepcopy(qmsetting_base)
qmsetting_dos['dos_mesh'] = [12, 12, 12]

In [4]:
# construct crystals based on cyl_map and si, gaas, cdse crystals

mol_base = []
#for cyl in ['si', 'gaas', 'cdse']:
for cyl in ['si', 'gaas']:
    name = 'xyz/%s.xyz' % cyl
    mol_base.append(qtk.Molecule(name))
mol_base

mols_grp = []
for mol in mol_base:
    mol_grp = [mol]
    for info in cyl_map[mol.name][1:]:
        new = mol.copy()
        new.name = info[0]
        new.setCelldm([info[1], info[1], info[1], 0, 0, 0])
        for i in range(new.N):
            if new.Z[i] != info[2][i]:
                new.setAtoms(i, Z=info[2][i])
        mol_grp.append(new)
    mols_grp.append(mol_grp)
    
mols_all = list(qtk.flatten(mols_grp))

ref_bnd_path = 'Eg_ref_vary_a_bnd_rst'
ref_dos_path = 'Eg_ref_vary_a_dos_rst'
bnd = 'Eg_prd_bnd'
dos = 'Eg_prd_dos'

In [6]:
inps_bnd = []
inps_dos = []
for mols in mols_grp:
    cyl_names = [_[0] for _ in cyl_map[mols[0].name]]
    for mol_ref in mols:
        for mol_tar in mols:
            for mol_rst in mols:
                
                mol = mol_tar.copy()
                mol.name = mol_tar.name + '_r-' + mol_rst.name + '_a-' + mol_ref.name
                
                # use ref as geometry scan
                ind = cyl_names.index(mol_ref.name)
                a0 = cyl_map[mols[0].name][ind][1]
                mol.setCelldm([a0, a0, a0, 0, 0, 0])
                
                # use reference system with target geometry
                rst_name = mol_rst.name + '_a-' + mol_ref.name
                rst_path_bnd = os.path.abspath(ref_bnd_path + '/' + rst_name)
                rst_path_dos = os.path.abspath(ref_dos_path + '/' + rst_name)

                # band scan restart
                inp_b = qtk.QMInp(
                    mol,
                    restart_wavefunction_path = rst_path_bnd,
                    restart_density_path = rst_path_bnd,
                    **qmsetting_bnd)
                inps_bnd.append(inp_b)

                # DOS restart
                inp_d = qtk.QMInp(
                    mol,
                    restart_wavefunction_path = rst_path_dos,
                    restart_density_path = rst_path_dos,
                    **qmsetting_dos)
                inps_dos.append(inp_d)
            
print len(inps_bnd)

1458
